# Main predictor: Coffee shops
1. Read data
2. Extract licenses with coffee or cafe in them
3. Extract year they were licensed
4. Convert area to Chicago Community Area

In [87]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/00_sourcedata')
col_list = [
    "ID",
    "LEGAL NAME",
    "DOING BUSINESS AS NAME",
    "ADDRESS",
    "CITY",
    "STATE",
    "ZIP CODE",
    "WARD",
    "PRECINCT",
    "WARD PRECINCT",
    "LICENSE STATUS",
    "LICENSE STATUS CHANGE DATE",
    "LICENSE DESCRIPTION",
    "BUSINESS ACTIVITY ID",
    "BUSINESS ACTIVITY",
    "LICENSE TERM START DATE",
    "LICENSE TERM EXPIRATION DATE",
    "DATE ISSUED",
    "LATITUDE",
    "LONGITUDE",
    'LOCATION'
]
df_full = pd.read_csv(os.getcwd()+
    "/Business_Licenses.csv",
    usecols=col_list,
    low_memory=False,
)

# https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr


## Cleaning for coffee shops and extracting year

In [88]:
df_full = df_full.loc[
    (df_full["STATE"] == "IL") & (df_full["CITY"] == "CHICAGO")
]  # subsetting for chicago
names_doingbusiness = [str(x).lower() for x in df_full["DOING BUSINESS AS NAME"]]
names_businessactivity = [str(x).lower() for x in df_full["BUSINESS ACTIVITY"]]
names_legalactivity = [str(x).lower() for x in df_full["LEGAL NAME"]]

keep_row = []
for a, b, c in zip(names_doingbusiness, names_businessactivity, names_legalactivity):
    keep = 0
    if (
        ("coffee" in a)
        or ("coffee" in b)
        or ("coffee" in c)
        or ("cafe" in a)
        or ("cafe" in b)
        or ("coffee" in c)
    ):
        keep = 1
    keep_row.append(keep)


df_full.insert(loc=0, column="names_doingbusiness", value=names_doingbusiness)
df_full.insert(loc=1, column="names_businessactivity", value=names_businessactivity)
df_full.insert(loc=2, column="names_legalactivity", value=names_legalactivity)
df_full.insert(loc=3, column="keep_row", value=keep_row)
df = df_full.loc[df_full.keep_row == 1]
df = df.drop(["LEGAL NAME", "DOING BUSINESS AS NAME", "CITY", "STATE"], axis=1)


In [89]:
df["lc_start_date"] = pd.to_datetime(df["LICENSE TERM START DATE"], format="%m/%d/%Y")
df['year']=df['lc_start_date'].map(lambda x: x.year)
#df.insert(loc=5, column='years', value=years)
df=df.loc[df.year>2000]
df = df.drop(
    [
        "LICENSE TERM START DATE",
        "LICENSE TERM EXPIRATION DATE",
        "DATE ISSUED",
        "LICENSE STATUS",
        "LICENSE STATUS CHANGE DATE",
    ],
    axis=1,
)
df.head()


,names_doingbusiness,names_businessactivity,names_legalactivity,keep_row,ID,ADDRESS,ZIP CODE,WARD,PRECINCT,WARD PRECINCT,LICENSE DESCRIPTION,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,LATITUDE,LONGITUDE,LOCATION,lc_start_date,year
201,arturo express,retail sales of perishable foods,arturo coffee inc,1,53931-20200916,130 S CANAL ST,60606,42.0,12.0,42-12,Retail Food Establishment,775,Retail Sales of Perishable Foods,41.879616,-87.639765,"(41.87961626801739, -87.63976452343368)",2020-09-16,2020.0
840,social junkie,"preparation of food, coffee or drinks",3848 n. harlem tavern llc,1,2551384-20171214,3848 N HARLEM AVE 1,60634,38.0,3.0,38-3,Retail Food Establishment,736,"Preparation of Food, Coffee or Drinks",41.950348,-87.807329,"(41.95034769269968, -87.80732944580947)",2017-12-14,2017.0
1223,social junkie,"preparation of food, coffee or drinks",3848 n. harlem tavern llc,1,2551384-20191216,3848 N HARLEM AVE 1,60634,38.0,3.0,38-3,Retail Food Establishment,736,"Preparation of Food, Coffee or Drinks",41.950348,-87.807329,"(41.95034769269968, -87.80732944580947)",2019-12-16,2019.0
2027,lutz continental cafe & pastry shop,retail sales of perishable foods,"pastry partners, inc.",1,1356917-20201216,2454-2458 W MONTROSE AVE,60618,47.0,21.0,47-21,Retail Food Establishment,775,Retail Sales of Perishable Foods,41.961409,-87.690796,"(41.96140869447139, -87.6907962429884)",2020-12-16,2020.0
2251,gaslight coffee roasters,retail sales of perishable foods,gaslight coffee roasters llc,1,2185009-20201016,2385 N MILWAUKEE AVE,60647,1.0,10.0,1-10,Retail Food Establishment,775,Retail Sales of Perishable Foods,41.924668,-87.699992,"(41.92466824748827, -87.69999236109766)",2020-10-16,2020.0


# Mapping to Community Areas

#### Option A: This code demonstrates how to take the shapefile provided on the City of Chicago Open Data Portal and converts it to a JSON file, with top-level keys named for each neighborhood, and associated values that give the latitude and longitude pairs corresponding to the neighborhood's boundaries. For example: (https://github.com/craigmbooth/chicago_neighborhood_finder)

#Fails to build GDAL

#### Option B: Determining Chicago neighborhood from latitude and longitude. The script uses a GeoJSON file containing polygons of Chicago neighborhoods. Once all of the polygons have been generated, a simple point in polygon algorithm is used to determine which neighborhood contains the point of interest. (https://github.com/jkgiesler/parse-chicago-neighborhoods/blob/master/gps_to_neighborhood.py)

In [90]:
df_loc=pd.DataFrame(df.LONGITUDE)
df_loc['LATITUDE']=pd.DataFrame(df.LATITUDE)

In [93]:
#df_loc.to_csv("Locations.csv")
## run gps_to_neighborhood.py 
locations= pd.read_csv("Locations_withCCA.csv")
locations=locations.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
locations.head()

,LONGITUDE,LATITUDE,CCA
0,-87.639765,41.879616,WEST LOOP
1,-87.807329,41.950348,DUNNING
2,-87.807329,41.950348,DUNNING
3,-87.690796,41.961409,LINCOLN SQUARE
4,-87.699992,41.924668,LOGAN SQUARE


In [94]:
locations['CCA']= np.where(locations.LATITUDE.isnull(),np.nan, locations['CCA'])

In [95]:
locations.shape

(15499, 3)

In [96]:
df.shape

(15499, 18)

In [98]:
df_new = pd.concat([df, locations], axis=1)

In [99]:
df_new.CCA.nunique()

74

In [102]:
df_new.head()

,names_doingbusiness,names_businessactivity,names_legalactivity,keep_row,ID,ADDRESS,ZIP CODE,WARD,PRECINCT,WARD PRECINCT,...,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,LATITUDE,LONGITUDE,LOCATION,lc_start_date,year,LONGITUDE,LATITUDE,CCA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,-87.639765,41.879616,WEST LOOP
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,-87.807329,41.950348,DUNNING
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,-87.807329,41.950348,DUNNING
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,-87.690796,41.961409,LINCOLN SQUARE
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,-87.699992,41.924668,LOGAN SQUARE


In [116]:
coffee_by_year_CCA=df_new[["year","CCA","keep_row"]].groupby(["CCA","year"],as_index=False).count()
coffee_by_year_CCA.head()

,CCA,year,keep_row
0,ANDERSONVILLE,2018.0,1
1,BACK OF THE YARDS,2021.0,1
2,"BELMONT CRAGIN,HERMOSA",2019.0,1
3,"BELMONT CRAGIN,HERMOSA",2020.0,1
4,BEVERLY,2019.0,1


In [117]:
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/02_processed_data')
coffee_by_year_CCA.to_csv("coffee_by_year_CCA.csv")